## Hyperparameter Tuning and Model Selection

In [12]:
# Standard packages
import pandas as pd
import numpy as np

from sklearn.model_selection import *
# KFold, train_test_split, cross_val_score, RandomizedSearchCV, ...
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
# mean_absolute_error, mean_squared_error, ...

import matplotlib.pyplot as plt
%matplotlib inline

# notebook settings
from IPython.display import display
pd.options.display.max_columns = None

# hide warnings
import warnings
warnings.filterwarnings("ignore")

In [13]:
# model-specific packages
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor

**Data ingestion, holdout split, and preparation**

In [14]:
# Read in data
above = '../../../'
data = pd.read_pickle(above + 'd_PCmd.pkl')

# Create holdout set and label X and Y columns.
# RANDOM STATE is 5, don't change
data_holdout = data.sample(frac = 0.2, random_state=5)
X_holdout = data_holdout.drop('s_sale_price', axis=1)
y_holdout = data_holdout['s_sale_price']

# Remove holdout set from training data and label X and Y columns
data_train = data.drop(data_holdout.index)
X = data_train.drop('s_sale_price', axis=1)
y = data_train['s_sale_price']

# Scale features (important since we are doing regression)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_holdout = scaler.transform(X_holdout)
# X_holdout = pd.DataFrame(scaler.fit_transform(X_holdout))

**Prepare subsampled data**

From our learning curve analysis, we saw that our target X_train subsample size is 2^13 = 8192.
In a 4-fold CV, where 25% would be set aside as test set, we need the total downsample size to then be 2^13/(.75) = 10.9k instances. (We can use the non-sampled instances for final validation as a holdout set, so no need to create a separate holdout set from the downsampled set.)

In [15]:
downsampled = data.sample(n = 10900, random_state = 5)

# separate downsampled set into X-Y for the CV.
X_downsampled = downsampled.drop('s_sale_price', axis=1)
y_downsampled = downsampled['s_sale_price']

**XGB Hyperparam tuning**


reference:
https://towardsdatascience.com/cross-validation-and-hyperparameter-tuning-how-to-optimise-your-machine-learning-model-13f005af9d7d

In [17]:
# Maximum number of levels in tree
xgb_max_depth = [int(x) for x in np.linspace(2, 20, 10)]

# Minimum number of instaces needed in each node
xgb_min_child_weight = [int(x) for x in np.linspace(1, 10, 10)]

# Tree construction algorithm used in XGBoost
xgb_tree_method = ['auto', 'exact', 'approx', 'hist', 'gpu_hist']

# Minimum loss reduction required to make further partition
xgb_gamma = [int(x) for x in np.linspace(0, 0.5, 6)]

# Learning objective used
xgb_objective = ['reg:squarederror', 'reg:squaredlogerror']

# Learning rate
# xgb_learning_rate = [x for x in np.linspace(0.1, 0.6, 6)]
xgb_learning_rate = [0.05, 0.1, 0.2, .3]

# Number of trees to be used
# xgb_n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]
xgb_n_estimators = [100, 200, 300]

# Subsample
xgb_subsample = [0.8, 0.9, 1.0]

# Create the grid
xgb_param_grid = {
                  #'max_depth': xgb_max_depth,
                  #'min_child_weight': xgb_min_child_weight,
                  #'tree_method': xgb_tree_method,
                  #'gamma': xgb_gamma,
                  #'objective': xgb_objective,
                  'n_estimators': xgb_n_estimators,
                  'learning_rate': xgb_learning_rate,
                  'subsample': xgb_subsample
                 }

display(xgb_param_grid)

{'n_estimators': [100, 200, 300],
 'learning_rate': [0.05, 0.1, 0.2, 0.3],
 'subsample': [0.8, 0.9, 1.0]}

In [18]:
xgb_regressor = XGBRegressor()

gs = GridSearchCV(estimator = xgb_regressor,
                  param_grid = xgb_param_grid,
                  scoring = 'r2',
                  cv = 4,
                  verbose = 1)

gs.fit(X_downsampled, y_downsampled)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:  6.9min finished


GridSearchCV(cv=4,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_para

In [20]:
print("Best parameter: ", gs.best_params_)
print("Best score: ", gs.best_score_)
print("Lowest Root Mean Square Error (RMSE): ", np.sqrt(np.abs(gs.best_score_)))

print(gs.cv_results_['params'])
print(gs.cv_results_['mean_test_score'])

Best parameter:  {'learning_rate': 0.05, 'n_estimators': 100, 'subsample': 0.9}
Best score:  0.6864510161913286
Lowest Root Mean Square Error (RMSE):  0.8285233950778509
[{'learning_rate': 0.05, 'n_estimators': 100, 'subsample': 0.8}, {'learning_rate': 0.05, 'n_estimators': 100, 'subsample': 0.9}, {'learning_rate': 0.05, 'n_estimators': 100, 'subsample': 1.0}, {'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 0.8}, {'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 0.9}, {'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 1.0}, {'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 0.8}, {'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 0.9}, {'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 1.0}, {'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 0.8}, {'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 0.9}, {'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0}, {'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.8}, {

In [21]:
# try again with the regular data, and see if the best params are different on the same range
gs.fit(X, y)

print("Best parameter: ", gs.best_params_)
print("Best score: ", gs.best_score_)

Fitting 4 folds for each of 36 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 50.3min finished


Best parameter:  {'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 0.9}
Best score:  0.7212719313445051


-------------

**Ridge Regressor Hyperparam Tuning**

In [67]:
ridge_alphas = [x for x in np.linspace(0.001, 20, 10)]

ridge_param_grid = {
                  'alpha': ridge_alphas
                 }

ridge = Ridge()

gs_ridge = GridSearchCV(estimator = ridge,
                        param_grid = ridge_param_grid,
                        scoring = 'r2',
                        cv = 4,
                        verbose = 3)

gs_ridge.fit(X, y)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.500, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.483, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.528, total=   0.0s
[CV] alpha=0.001 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ......................... alpha=0.001, score=0.385, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.502, total=   0.1s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.484, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.529, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.394, total=   0.0s
[CV] alpha=4.445222222222222 .........................................
[CV] ............. alpha=4.445222222222222, score=0.504, total=   0.1s
[CV] alpha=4.445222222222222 .........................................
[CV] ............. alpha=4.445222222222222, score=0.485, total=   0.0s
[CV] alpha=4.445222222222222 .........................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.9s finished


GridSearchCV(cv=4, estimator=Ridge(),
             param_grid={'alpha': [0.001, 2.223111111111111, 4.445222222222222,
                                   6.667333333333333, 8.889444444444443,
                                   11.111555555555555, 13.333666666666664,
                                   15.555777777777775, 17.77788888888889,
                                   20.0]},
             scoring='r2', verbose=3)

In [69]:
print("Best parameter: ", gs_ridge.best_params_)
print("Best score: ", gs_ridge.best_score_)
print("Lowest Root Mean Square Error (RMSE): ", np.sqrt(np.abs(gs_ridge.best_score_)))

print(gs_ridge.cv_results_['params'])
print(gs_ridge.cv_results_['mean_test_score'])

Best parameter:  {'alpha': 20.0}
Best score:  0.48541064914073195
Lowest Root Mean Square Error (RMSE):  0.6967141803786772
[{'alpha': 0.001}, {'alpha': 2.223111111111111}, {'alpha': 4.445222222222222}, {'alpha': 6.667333333333333}, {'alpha': 8.889444444444443}, {'alpha': 11.111555555555555}, {'alpha': 13.333666666666664}, {'alpha': 15.555777777777775}, {'alpha': 17.77788888888889}, {'alpha': 20.0}]
[0.47394163 0.47741175 0.47961312 0.48114422 0.4822795  0.48316243
 0.48387504 0.4844675  0.48497213 0.48541065]


In [73]:
# try again with the subsampled data, and see if the best params are different on the same range
gs_ridge.fit(X_downsampled, y_downsampled)

print("Best parameter: ", gs.best_params_)
print("Best score: ", gs.best_score_)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.677, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.555, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.464, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ......................... alpha=0.001, score=0.464, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.677, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ............. alpha=2.223111111111111, score=0.555, total=   0.0s
[CV] alpha=2.223111111111111 .........................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ............. alpha=8.889444444444443, score=0.677, total=   0.0s
[CV] alpha=8.889444444444443 .........................................
[CV] ............. alpha=8.889444444444443, score=0.555, total=   0.0s
[CV] alpha=8.889444444444443 .........................................
[CV] ............. alpha=8.889444444444443, score=0.465, total=   0.0s
[CV] alpha=8.889444444444443 .........................................
[CV] ............. alpha=8.889444444444443, score=0.464, total=   0.0s
[CV] alpha=11.111555555555555 ........................................
[CV] ............ alpha=11.111555555555555, score=0.677, total=   0.0s
[CV] alpha=11.111555555555555 ........................................
[CV] ............ alpha=11.111555555555555, score=0.555, total=   0.0s
[CV] alpha=11.111555555555555 ........................................
[CV] ............ alpha=11.111555555555555, score=0.465, total=   0.0s
[CV] alpha=11.111555555555555 ........................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.5s finished


**Final Model Eval**

In [72]:
# Create Ridge Regressor with tuned hyperparams
ridge_tuned = Ridge(alpha = 20)

# Create XGBRegressor with tuned hyperparams
# {'learning_rate': 0.05, 'n_estimators': 200, 'subsample': 0.9}
xgb_tuned = XGBRegressor(n_estimators = 200,
                         learning_rate = 0.05,
                         subsample = 0.9)

# Train the models using 80% of the original data
ridge_tuned.fit(X, y)
xgb_tuned.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [103]:
# Define a function that compares all final models
def final_comparison(models, test_features, test_labels):
    scores = pd.DataFrame()
    
    for model in models:
        predictions = model.predict(test_features)
        
        mae = mean_absolute_error(test_labels, predictions)
        mse = mean_squared_error(test_labels, predictions) 
        r2 = r2_score(test_labels, predictions)
        errors = abs(predictions - test_labels)
        mape = 100 * np.mean(errors / test_labels)
        accuracy = 100 - mape
        
        scores[str(model)] = ["%.3f" % mae, "%.3f" % mse, "%.3f" % r2, "%.3f" % accuracy]
    
    scores.index = ['Mean Absolute Error', 'Mean Squared Error', 'R^2', 'Accuracy']
    return scores

In [104]:
# Call the comparison function with the final optimized models
final_scores = final_comparison([ridge_tuned, xgb_tuned], X_holdout, y_holdout)

# Adjust the column headers
final_scores.columns  = ['Ridge Regressor', 'Extreme Gradient Boosting']

In [105]:
display(final_scores)

,Ridge Regressor,Extreme Gradient Boosting
Mean Absolute Error,273626.825,179484.760
Mean Squared Error,780822377030.766,387433500723.288
R^2,0.530,0.767
Accuracy,58.068,69.583
